In [ ]:
# Install PyTorch
!pip install pytorch torchvision torchaudio #was conda install here?
# Install transformers library
!pip install transformers==4.30
# Install other required packages
!pip install langchain sentence_transformers huggingface-hub

  Using cached pytorch-1.0.2.tar.gz (689 bytes)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
ERROR: Could not build wheels for pytorch, which is required to install pyproject.toml-based projects
  Using cached transformers-4.30.0-py3-none-any.whl (7.2 MB)
  Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Succ

In [ ]:
!pip install pypdf
!pip install -U langchain-community
!pip install bitsandbytes
!pip install faiss-cpu langchain-openai tiktoken unstructured selenium newspaper3k textstat

In [ ]:
!pip install accelerate

In [ ]:
from google.colab import drive, userdata
import os
import pickle
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import pipeline
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/MyDrive/WAI_project/"
loader = DirectoryLoader(f"{root_dir}", glob="*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()

# Loading each page as a separate document rather than 2 documents?

Mounted at /content/gdrive


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# you will need to enable GPU usage on your session for this

Using device: cuda


In [ ]:
hf_token= userdata.get('HF_TOKEN')
os.environ["HF_TOKEN"] = hf_token # you will be referencing your own HF token

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
quantization_config = BitsAndBytesConfig(load_in_4bit=True)
model = AutoModelForCausalLM.from_pretrained("google/gemma-1.1-7b-it", quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained("google/gemma-1.1-7b-it", quantization_config=quantization_config)

# Instantiating model and tokeniser (converts query sent to llm to a vector)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# Set environment variables for OpenAI
os.environ["OPENAI_API_KEY"] = "OPEN AI KEY"

In [ ]:
# Import libraries
import os
from google.colab import drive, userdata
import pickle
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoModel
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import pipeline
from google.colab import drive, userdata

In [ ]:
## Step 1: Ingestion and Chunking
# Initialize text_splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
)

def ingest_and_chunk_pdfs():
    """
    Ingests PDF documents and chunks the content into smaller segments.

    Returns:
    - list: List of text chunks.
    """

    loader = DirectoryLoader(f"{root_dir}", glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()

    # Debug: Print the number of documents loaded
    print(f"Number of documents loaded: {len(documents)}")
    for i, doc in enumerate(documents[:3]):  # Print first 3 documents for verification
        print(f"Document {i+1}:")
        print(doc.page_content[:500])  # Print first 500 characters of each document

    texts = text_splitter.split_documents(documents)

    # Debug: Print the number of text chunks created
    print(f"Number of text chunks: {len(texts)}")
    for i, text in enumerate(texts[:3]):  # Print first 3 chunks for verification
        print(f"Chunk {i+1}:")
        print(text.page_content[:500])  # Print first 500 characters of each chunk
    return texts

texts = ingest_and_chunk_pdfs()

Number of documents loaded: 645
Document 1:
Centre Number Candidate NumberWrite your name here
Surname Other names
Total MarksPaper Reference
Turn over     Pearson 
Edexcel GCSE
*P44728A0112* P44728A
©2015 Pearson Education Ltd.
4/1/1/1Instructions
• Use black  ink or ball-point pen.• Fill in the boxes  at the top of this page with your name,  
 centre number and candidate number.• Answer all questions.•  Answer the questions in the spaces provided  
– there may be more space than you need .
Information
• The total mark for this paper is 
Document 2:
2*P44728A0212*Answer ALL questions.
1 Study Section 1 (pages 2, 3 and 4) of the Resource Booklet and answer the following 
questions. 
 (a) (i) Define the term aquifer .
(2)
............................................................................................................................... ............................................................................................................................... .............

In [ ]:
"""
from InstructorEmbedding import INSTRUCTOR

embedding_store_path = os.path.join(root_dir, "embedding_store")

def store_embeddings(docs, embeddings, store_name, path):
    vector_store = FAISS.from_documents(docs, embeddings)
    with open(os.path.join(path, f"faiss_{store_name}.pkl"), "wb") as f:
        pickle.dump(vector_store, f)

def load_embeddings(store_name, path):
    with open(os.path.join(path, f"faiss_{store_name}.pkl"), "rb") as f:
        vector_store = pickle.load(f)
    return vector_store

# Load the model and tokenizer
model_name = "hkunlp/instructor-xl"
model = AutoModel.from_pretrained(model_name, use_auth_token=os.getenv("HF_TOKEN"))
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=os.getenv("HF_TOKEN"))
"""

'\nfrom InstructorEmbedding import INSTRUCTOR\n\nembedding_store_path = os.path.join(root_dir, "embedding_store")\n\ndef store_embeddings(docs, embeddings, store_name, path):\n    vector_store = FAISS.from_documents(docs, embeddings)\n    with open(os.path.join(path, f"faiss_{store_name}.pkl"), "wb") as f:\n        pickle.dump(vector_store, f)\n\ndef load_embeddings(store_name, path):\n    with open(os.path.join(path, f"faiss_{store_name}.pkl"), "rb") as f:\n        vector_store = pickle.load(f)\n    return vector_store\n\n# Load the model and tokenizer\nmodel_name = "hkunlp/instructor-xl"\nmodel = AutoModel.from_pretrained(model_name, use_auth_token=os.getenv("HF_TOKEN"))\ntokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=os.getenv("HF_TOKEN"))\n'

In [ ]:
def generate_embeddings(texts):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    # Example: Use mean pooling to get sentence embeddings
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

In [ ]:
def store_embeddings(docs, embeddings, store_name, path):
    # Generate embeddings for the documents
    doc_embeddings = generate_embeddings([doc.page_content for doc in docs])

    # Create a FAISS vector store from the document embeddings
    vector_store = FAISS.from_documents(docs, embeddings)

    # Save the vector store to a file
    with open(os.path.join(path, f"faiss_{store_name}.pkl"), "wb") as f:
        pickle.dump(vector_store, f)

In [ ]:
def load_embeddings(store_name, path):
    with open(os.path.join(path, f"faiss_{store_name}.pkl"), "rb") as f:
        vector_store = pickle.load(f)
    return vector_store

def retrieve_relevant_chunks(question, vector_store, num_chunks=5):
    # Retrieve similar chunks from the vector store
    docs = vector_store.similarity_search(question, k=num_chunks)
    return docs

In [ ]:
"""
from langchain.embeddings import HuggingFaceInstructEmbeddings

def initialize_huggingface_embeddings(model_name="hkunlp/instructor-xl", device="cuda"):
    return HuggingFaceInstructEmbeddings(model_name=model_name, model_kwargs={"device": device})

# Initialize embeddings
instructor_embeddings = initialize_huggingface_embeddings()
"""

'\nfrom langchain.embeddings import HuggingFaceInstructEmbeddings\n\ndef initialize_huggingface_embeddings(model_name="hkunlp/instructor-xl", device="cuda"):\n    return HuggingFaceInstructEmbeddings(model_name=model_name, model_kwargs={"device": device})\n\n# Initialize embeddings\ninstructor_embeddings = initialize_huggingface_embeddings()\n'

In [ ]:
"""
def retrieve_relevant_chunks(question, vector_store, num_chunks=5):
    # Retrieve similar chunks from the vector store
    docs = vector_store.similarity_search(question, k=num_chunks)
    return docs
"""
def format_prompt(question, chunks):
    context = "\n".join([chunk.page_content for chunk in chunks])
    prompt = f"Provide an answer to the following question using only the context provided: {question}? " \
             f"If you cannot answer this question from the information provided, respond with 'There is insufficient information to answer this question.'\n\n{context}"
    return prompt

"""
def gen_answer(prompt, max_length=200):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    response = model.generate(input_ids, max_new_tokens=max_length)
    answer = tokenizer.decode(response[0], skip_special_tokens=True)
    return answer.strip()
"""
"""
def gen_answer(prompt, model_name='llm', max_length=200, temperature=0.7):
    generator = pipeline('text-generation', model=model_name)
    response = generator(prompt, max_length=max_length, temperature=temperature, num_return_sequences=1)
    return response[0]['generated_text'].strip()
"""

"\ndef gen_answer(prompt, model_name='llm', max_length=200, temperature=0.7):\n    generator = pipeline('text-generation', model=model_name)\n    response = generator(prompt, max_length=max_length, temperature=temperature, num_return_sequences=1)\n    return response[0]['generated_text'].strip()\n"

In [ ]:
"""def gen_answer(prompt, max_length=200, temperature=0.7):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    response = model.generate(input_ids, max_new_tokens=max_length, temperature=temperature)
    answer = tokenizer.decode(response[0], skip_special_tokens=True)
    return answer.strip()"""
def gen_answer(prompt, max_length=200, temperature=0.7):
    inputs = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    response = model.generate(inputs, max_new_tokens=max_length, temperature=temperature)
    answer = tokenizer.decode(response[0], skip_special_tokens=True)
    return answer.strip()


In [ ]:
def main(question):
    # Load your vector store (ensure it's already populated with embeddings)
    vector_store = load_embeddings("embedding_store", root_dir)

    # Retrieve relevant chunks based on the question
    relevant_chunks = retrieve_relevant_chunks(question, vector_store)

    # Format the prompt for the LLM
    prompt = format_prompt(question, relevant_chunks)

    # Generate the answer using the LLM
    answer = gen_answer(prompt)
    return answer

# Example usage
question = "What are the main causes of climate change?"
answer = main(question)
print(answer)


FileNotFoundError: [Errno 2] No such file or directory: '/content/gdrive/MyDrive/WAI_project/faiss_embedding_store.pkl'

In [ ]:
def main(question):
    # Load your vector store (ensure it's already populated with embeddings)
    vector_store = load_embeddings("embedding_store", root_dir)

    # Retrieve relevant chunks based on the question
    relevant_chunks = retrieve_relevant_chunks(question, vector_store)

    # Format the prompt for the LLM
    prompt = format_prompt(question, relevant_chunks)

    # Generate the answer using the LLM
    answer = gen_answer(prompt)
    return answer


In [ ]:
# Example usage
question = "What are the main causes of climate change?"
answer = main(question)
print(answer)